In [1]:
# from google.colab import files
# from google.colab import drive 
# drive.mount("/drive")
# uploaded = files.upload()

In [2]:
import numpy as np 
import pandas as pd

from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

In [3]:
training_variantsDF = pd.read_csv("training_variants")
training_variantsDF.head(5)

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4


In [4]:
training_textDF = pd.read_csv("training_text", sep="\|\|", engine='python',header=None, skiprows=1, names=["ID","Text"])
training_textDF.head(5)

,ID,Text
0,0,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,Abstract Background Non-small cell lung canc...
2,2,Abstract Background Non-small cell lung canc...
3,3,Recent evidence has demonstrated that acquired...
4,4,Oncogenic mutations in the monomeric Casitas B...


In [5]:
training_mergeDF = training_variantsDF.merge(training_textDF,left_on="ID",right_on="ID")

In [6]:
training_mergeDF.head(5)

,ID,Gene,Variation,Class,Text
0,0,FAM58A,Truncating Mutations,1,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,CBL,W802*,2,Abstract Background Non-small cell lung canc...
2,2,CBL,Q249E,2,Abstract Background Non-small cell lung canc...
3,3,CBL,N454D,3,Recent evidence has demonstrated that acquired...
4,4,CBL,L399V,4,Oncogenic mutations in the monomeric Casitas B...


In [7]:
testing_variantsDF = pd.read_csv("test_variants")
testing_variantsDF.head(5)

,ID,Gene,Variation
0,0,ACSL4,R570S
1,1,NAGLU,P521L
2,2,PAH,L333F
3,3,ING1,A148D
4,4,TMEM216,G77A


In [8]:
testing_textDF = pd.read_csv("test_text", sep="\|\|", engine='python',header=None, skiprows=1, names=["ID","Text"])
testing_textDF.head(5)

,ID,Text
0,0,2. This mutation resulted in a myeloproliferat...
1,1,Abstract The Large Tumor Suppressor 1 (LATS1)...
2,2,Vascular endothelial growth factor receptor (V...
3,3,Inflammatory myofibroblastic tumor (IMT) is a ...
4,4,Abstract Retinoblastoma is a pediatric retina...


In [9]:
testing_mergeDF = testing_variantsDF.merge(testing_textDF,left_on="ID",right_on="ID")
testing_mergeDF.head(5)

,ID,Gene,Variation,Text
0,0,ACSL4,R570S,2. This mutation resulted in a myeloproliferat...
1,1,NAGLU,P521L,Abstract The Large Tumor Suppressor 1 (LATS1)...
2,2,PAH,L333F,Vascular endothelial growth factor receptor (V...
3,3,ING1,A148D,Inflammatory myofibroblastic tumor (IMT) is a ...
4,4,TMEM216,G77A,Abstract Retinoblastoma is a pediatric retina...


In [10]:
training_mergeDF['Text'] = training_mergeDF.apply(lambda row: row['Gene'] if pd.isnull(row['Text']) else row['Text'],
    axis=1
)

In [11]:
y=training_mergeDF.Class
X=training_mergeDF[["Text","Variation",]]

In [12]:
X.head()

,Text,Variation
0,Cyclin-dependent kinases (CDKs) regulate a var...,Truncating Mutations
1,Abstract Background Non-small cell lung canc...,W802*
2,Abstract Background Non-small cell lung canc...,Q249E
3,Recent evidence has demonstrated that acquired...,N454D
4,Oncogenic mutations in the monomeric Casitas B...,L399V


In [13]:
y.head()

0    1
1    2
2    2
3    3
4    4
Name: Class, dtype: int64

In [14]:
from sklearn.model_selection  import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [15]:
X_train

,Text,Variation
285,Poorly differentiated thyroid cancer (PDTC) an...,A113_splice
2723,Genes of the Raf family encode kinases that ar...,G464R
3042,Amplifications and mutations in the KIT proto-...,W557_V560del
1171,We previously reported four PIK3CA mutations i...,E542G
91,Multiple self-healing squamous epithelioma (MS...,C41Y
...,...,...
2763,Kinase inhibitors are accepted treatment for m...,K601E
905,Purpose: Platelet-derived growth factor recept...,I834V
1096,Langerhans cell histiocytosis (LCH) is now und...,T779fs
235,Non鈥搒mall-cell lung cancer is the leading caus...,T790M


In [16]:
X_test

,Text,Variation
3212,Retinoblastoma is a pediatric tumor of the dev...,Truncating Mutations
926,"To determine the potential of crenolanib, a po...",560_561insER
813,The increasing application of gene panels for ...,R487W
1177,Some clinical findings have suggested that sys...,N1068fs*4
990,More than 50% of transitional cell carcinomas ...,F158C
...,...,...
1488,"Signaling through FGF receptors, which constit...",C382R
553,Hub proteins are connected through binding int...,Y298A
1926,Basal activities of adenylate cyclase and gua...,L412F
2126,This is a comprehensive whole-genome/whole-exo...,C47S


In [17]:
y_train

285     7
2723    7
3042    2
1171    6
91      4
       ..
2763    7
905     3
1096    1
235     7
1061    2
Name: Class, Length: 2490, dtype: int64

In [18]:
y_test

3212    1
926     7
813     1
1177    7
990     4
       ..
1488    7
553     1
1926    7
2126    7
2479    4
Name: Class, Length: 831, dtype: int64

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import classification_report

In [20]:
vect_text= CountVectorizer(stop_words ='english')
vect_variation= CountVectorizer(stop_words ='english')

In [21]:
vect_text.fit(X_train["Text"])
vect_text.fit(X_test["Text"])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [22]:
vect_variation.fit(X_train["Variation"])
vect_variation.fit(X_test["Variation"])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [23]:
variation_tranform_train=vect_variation.transform(X_train["Variation"])
variation_tranform_test=vect_variation.transform(X_test["Variation"])

In [24]:
text_transformed_train= vect_text.transform(X_train["Text"])
text_transformed_test=vect_text.transform(X_test["Text"])

In [25]:
import scipy.sparse as sp
x_train_final = sp.hstack((variation_tranform_train,text_transformed_train))
x_test_final = sp.hstack((variation_tranform_test,text_transformed_test))

In [26]:
svc_model=svm.LinearSVC(C=1.0,dual=False, max_iter=1000)
svc_model.fit(x_train_final,y_train)

D:\AnacondaInstalled\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [27]:
y_pred_class = svc_model.predict(x_test_final)

In [28]:
y_pred_class

array([1, 7, 1, 2, 4, 4, 1, 2, 4, 1, 4, 4, 6, 1, 5, 1, 4, 4, 5, 2, 7, 3,
       5, 1, 4, 7, 2, 7, 1, 4, 1, 4, 7, 4, 7, 7, 4, 3, 7, 7, 7, 6, 4, 4,
       1, 7, 2, 7, 1, 3, 5, 4, 1, 7, 7, 3, 1, 4, 4, 1, 7, 6, 3, 7, 7, 4,
       4, 4, 4, 7, 1, 7, 7, 7, 1, 1, 7, 7, 7, 6, 6, 6, 4, 1, 4, 1, 4, 4,
       1, 7, 7, 7, 7, 5, 4, 7, 7, 2, 4, 7, 2, 6, 5, 1, 4, 7, 4, 5, 4, 7,
       5, 4, 2, 7, 7, 5, 6, 4, 1, 6, 7, 7, 2, 6, 1, 2, 5, 7, 4, 7, 4, 1,
       1, 7, 6, 7, 4, 6, 5, 4, 2, 1, 1, 7, 2, 2, 9, 7, 7, 7, 4, 2, 1, 2,
       6, 2, 7, 5, 1, 5, 4, 7, 4, 7, 6, 5, 7, 7, 1, 6, 3, 2, 7, 2, 1, 7,
       2, 2, 5, 7, 7, 5, 5, 2, 7, 4, 2, 7, 1, 4, 1, 2, 2, 5, 5, 4, 5, 7,
       5, 4, 4, 7, 7, 7, 6, 7, 7, 8, 9, 2, 1, 4, 7, 7, 3, 1, 4, 5, 7, 6,
       4, 2, 2, 7, 6, 5, 7, 7, 7, 4, 6, 4, 7, 4, 1, 7, 2, 1, 4, 7, 7, 7,
       1, 7, 7, 5, 7, 7, 9, 4, 1, 7, 4, 7, 7, 4, 1, 1, 4, 7, 7, 1, 1, 6,
       1, 7, 7, 7, 9, 7, 7, 7, 4, 6, 4, 7, 7, 7, 7, 4, 5, 4, 5, 5, 4, 1,
       7, 6, 7, 6, 1, 4, 1, 5, 4, 1, 2, 2, 4, 6, 7,

In [29]:
y_pred_class_df=pd.DataFrame(y_pred_class)
y_pred_class_df

,0
0,1
1,7
2,1
3,2
4,4
...,...
826,7
827,1
828,7
829,7


In [30]:
testing_mergeDF['predicted_class'] = y_pred_class_df

In [31]:
testing_mergeDF.head(5)

,ID,Gene,Variation,Text,predicted_class
0,0,ACSL4,R570S,2. This mutation resulted in a myeloproliferat...,1.0
1,1,NAGLU,P521L,Abstract The Large Tumor Suppressor 1 (LATS1)...,7.0
2,2,PAH,L333F,Vascular endothelial growth factor receptor (V...,1.0
3,3,ING1,A148D,Inflammatory myofibroblastic tumor (IMT) is a ...,2.0
4,4,TMEM216,G77A,Abstract Retinoblastoma is a pediatric retina...,4.0


In [32]:
onehot = pd.get_dummies(testing_mergeDF['predicted_class'])
testing_mergeDF = testing_mergeDF.join(onehot)

In [33]:
testing_mergeDF.rename(columns={'1.0':'1','2.0':'2','3.0':'3','4.0':'4','5.0':'5','6.0':'6','7.0':'7','8.0':'8','9.0':'9'},inplace = True)
testing_mergeDF.head(5)

,ID,Gene,Variation,Text,predicted_class,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
0,0,ACSL4,R570S,2. This mutation resulted in a myeloproliferat...,1.0,1,0,0,0,0,0,0,0,0
1,1,NAGLU,P521L,Abstract The Large Tumor Suppressor 1 (LATS1)...,7.0,0,0,0,0,0,0,1,0,0
2,2,PAH,L333F,Vascular endothelial growth factor receptor (V...,1.0,1,0,0,0,0,0,0,0,0
3,3,ING1,A148D,Inflammatory myofibroblastic tumor (IMT) is a ...,2.0,0,1,0,0,0,0,0,0,0
4,4,TMEM216,G77A,Abstract Retinoblastoma is a pediatric retina...,4.0,0,0,0,1,0,0,0,0,0


In [34]:
submission_df = testing_mergeDF[["ID",1,2,3,4,5,6,7,8,9]]
submission_df.columns = ['ID', 'class1','class2','class3','class4','class5','class6','class7','class8','class9']
submission_df.head(5)

,ID,class1,class2,class3,class4,class5,class6,class7,class8,class9
0,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1,0,0
2,2,1,0,0,0,0,0,0,0,0
3,3,0,1,0,0,0,0,0,0,0
4,4,0,0,0,1,0,0,0,0,0


In [35]:
submission_df.to_csv('submission.csv', index=False)